<a href="https://colab.research.google.com/github/prateekgulati/-Breast-Cancer-ML/blob/master/Notebook_Prateek/RG2_Notebook7_DavidNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Summary


This notebook has experiments to find good augmentation strategy to incorporate within the network.  
Augmentation-channel level: 
```
       One strategy randomly selected
          0: random_pad_crop_image(channel,padding=padding)
          1: flip_left_right(channel)
          2: cutout_channel(channel,size=cutSize)
          3: flip_left_right(random_pad_crop_image(channel,padding=padding))
          4: cutout_channel(random_pad_crop_image(channel,padding=padding),size=cutSize)
          5: cutout_channel(flip_left_right(channel),size=cutSize)
          6: cutout_channel(flip_left_right(random_pad_crop_image(channel,padding=padding)),size=cutSize)
```

|    Parameters   |     Applied at     | Train  | Test (epoch) |
|:---------------:|:------------------:|:------:|:------------:|
| pad=1,cutSize=4 | Initial Conv block |  99.73 | **90.95 (22nd)** |
| pad=1,cutSize=8 | Initial Conv block |  96.78 | **90.91 (24th)** |
| pad=2,cutSize=4 | Initial Conv block |  97.39 | 85.16 (23rd) |
| pad=2,cutSize=8 | Initial Conv block |  94.95 | 84.48 (24th) |
| pad=4,cutSize=4 | Initial Conv block |  99.20 | **91.49 (23rd)** |
| pad=4,cutSize=8 | Initial Conv block |  97.90 | **90.96 (24th)** |
| pad=1,cutSize=4 |   ResNet Block 1   |  97.44 | 84.21 (20th) |
| pad=1,cutSize=8 |   ResNet Block 1   |  93.17 | 83.63 (21st) |
| pad=2,cutSize=4 |   ResNet Block 1   |  94.19 | 83.68 (22nd) |
| pad=2,cutSize=8 |   ResNet Block 1   |  98.09 | **91.94 (24th)**    |
| pad=2,cutSize=2 |   ResNet Block 1   |  96.86 | 84.01 (18th) |
| pad=1,cutSize=2 |   ResNet Block 2   |  98.07 | 80.64 (16th) |
| pad=1,cutSize=4 |   ResNet Block 2   |  99.18 | **91.02 (24th)** |
| pad=2,cutSize=1 |   ResNet Block 2   |  97.71 |   78.54 (10th) |
| pad=2,cutSize=2 |   ResNet Block 2   |  99.74 | **90.46 (22nd)** |
| pad=2,cutSize=4 |   ResNet Block 2   |  93.46 | 81.31 (14th) |
| pad=1,cutSize=1 |   ResNet Block 3   |  100.0 | 82.16 (23nd) |
| pad=1,cutSize=2 |   ResNet Block 3   |  99.98 | 88.55 (24th) |
| pad=2,cutSize=1 |   ResNet Block 3   |  99.94 | 81.05 (24th) |
| pad=2,cutSize=2 |   ResNet Block 3   |  99.65 | 80.89 (24th) |
| pad=2,cutSize=4 |   ResNet Block 3   |  58.59 | 78.26 (24th) |

### Code



In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import random

In [0]:
tf.enable_eager_execution()


In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class Conv(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)    

  def call(self, inputs):
    return tf.nn.relu(self.conv(inputs))

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

test_mean = np.mean(x_train, axis=(0,1,2))
test_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
normalize_test = lambda x: ((x - test_mean) / test_std).astype('float32') # todo: check here
# pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

# x_train = normalize(pad4(x_train))
x_train = normalize(x_train)
# x_test = normalize(x_test)
x_test = normalize_test(x_test)

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.8*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.1*LEARNING_RATE, 0.005])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
def cutout_channel(img, prob=100, size=8, min_size=5, use_fixed_size=True):
  return tf.cond(tf.random.uniform([], 0, 100) > prob, lambda: img , lambda: get_cutout_channel(img,size,min_size,use_fixed_size))


def get_cutout_channel(img, size=8,min_size=2,use_fixed_size=True):
    height = tf.shape(img)[0]
    width = tf.shape(img)[1]
    channel = tf.shape(img)[2]
    area = tf.cast(width*height, tf.float32)
    if (use_fixed_size==True):
      s=size
    else:  
      s=tf.random.uniform([], min_size, size, tf.int32)
    x1 = tf.random.uniform([], 0, height+1-s , tf.int32) # get the x offset from top left
    y1 = tf.random.uniform([], 0, width+1-s , tf.int32)
    img1 = tf.ones_like(img)  
    #print(tf.shape(img1))
    cut_slice = tf.slice(
    img1,
    [x1, y1, 0],
    [s, s, channel])
    #create mask similar in shape to input image with cutout area having ones and rest of the area padded with zeros 
    mask = tf.image.pad_to_bounding_box(
      cut_slice,
      x1,
      y1,
      height,
      width
    )
    mask = tf.ones_like(mask) - mask
    tmp_img = tf.multiply(img,mask)
    cut_img =tmp_img
    return cut_img

In [0]:
def random_pad_crop_image(image,padding=4):
  # global ctr
  # ctr=ctr+1
  shape=tf.shape(image)  
  image=tf.pad(image,[(padding, padding), (padding, padding), (0, 0)])
  image=tf.image.random_crop(image,size=shape)
  return image

def random_pad_crop_batch(batch,padding=4):
  # global ctr
  # ctr=ctr+1
  shape=tf.shape(batch)  
  batch=tf.pad(batch,[(0, 0), (padding, padding), (padding, padding), (0, 0)])
  batch=tf.image.random_crop(batch,size=shape)
  return batch

def flip_left_right(image):
#   global ctr
#   ctr=ctr+1
  return tf.image.random_flip_left_right(image)  

def no_augmentation(batch):
  # global ctr
  # ctr=ctr+1
  return batch

def augmentDictBatch(batch,padding):
  return tf.switch_case(
        tf.random_uniform([],0,6, dtype=tf.dtypes.int32), 
        branch_fns={
            0: lambda:random_pad_crop_batch(batch,padding=padding[0]),
            1: lambda:flip_left_right(batch),
            2: lambda:flip_left_right(random_pad_crop_batch(batch,padding=padding[0])),
            3: lambda:random_pad_crop_batch(batch,padding=padding[1]),
            4: lambda:flip_left_right(random_pad_crop_batch(batch,padding=padding[1])),
        }, 
        default= lambda: batch # 5
    )  

def augmentDictChannel(batch,padding,cutSize):
  return tf.switch_case(
        tf.random_uniform([],0,9, dtype=tf.dtypes.int32), 
        branch_fns={
            0: lambda:tf.map_fn(lambda channel: random_pad_crop_image(channel,padding=padding), batch),
            1: lambda:tf.map_fn(lambda channel: flip_left_right(channel), batch),
            2: lambda:tf.map_fn(lambda channel: cutout_channel(channel,size=cutSize), batch),
            3: lambda:tf.map_fn(lambda channel: flip_left_right(random_pad_crop_image(channel,padding=padding)), batch),            
            4: lambda:tf.map_fn(lambda channel: cutout_channel(random_pad_crop_image(channel,padding=padding),size=cutSize), batch),
            5: lambda:tf.map_fn(lambda channel: cutout_channel(flip_left_right(channel),size=cutSize), batch),
            6: lambda:tf.map_fn(lambda channel: cutout_channel(flip_left_right(random_pad_crop_image(channel,padding=padding)),size=cutSize), batch),
        }, 
        default= lambda: batch # 7,8
    )

### Without Augmentation

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  training=True
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.6341621618652344 train acc: 0.42006 val loss: 1.2786644775390625 val acc: 0.5336 time: 84.12477540969849



epoch: 2 lr: 0.16 train loss: 1.0550838256835937 train acc: 0.62662 val loss: 1.0723008911132812 val acc: 0.6166 time: 168.25910019874573



epoch: 3 lr: 0.24 train loss: 0.7666204714965821 train acc: 0.73398 val loss: 0.8635984420776367 val acc: 0.6925 time: 252.41594672203064



epoch: 4 lr: 0.32 train loss: 0.5686230340576172 train acc: 0.80822 val loss: 0.794242236328125 val acc: 0.7234 time: 336.4668564796448



epoch: 5 lr: 0.4 train loss: 0.40912643966674805 train acc: 0.86958 val loss: 0.8083009780883789 val acc: 0.7189 time: 420.6586241722107


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.0018887103962898254 train acc: 1.0 val loss: 0.6530312286376954 val acc: 0.8096 time: 1681.9618873596191



epoch: 21 lr: 0.026000000000000002 train loss: 0.001757127342224121 train acc: 1.0 val loss: 0.6537623336791992 val acc: 0.8095 time: 1766.0287747383118



epoch: 22 lr: 0.019000000000000003 train loss: 0.0016434865772724152 train acc: 1.0 val loss: 0.6562681594848633 val acc: 0.8101 time: 1850.128155708313



epoch: 23 lr: 0.012 train loss: 0.0015578745555877686 train acc: 1.0 val loss: 0.6610282562255859 val acc: 0.8099 time: 1934.2064535617828



epoch: 24 lr: 0.005 train loss: 0.001461819063425064 train acc: 1.0 val loss: 0.6627364303588867 val acc: 0.8099 time: 2018.2653663158417


### Augmentation inside network
Augmentation Appled at: After 1st convolution  
Augmentation Strategy: Random Pad Crop (pad=1,cutSize=4)

In [0]:
pad=1
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch   

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.478090366821289 train acc: 0.4648 val loss: 1.1969029052734375 val acc: 0.5931 time: 336.0568616390228



epoch: 2 lr: 0.16 train loss: 0.7908851968383789 train acc: 0.71798 val loss: 0.8535251312255859 val acc: 0.7124 time: 660.4499454498291



epoch: 3 lr: 0.24 train loss: 0.5993705606079102 train acc: 0.79074 val loss: 1.1467464385986328 val acc: 0.6698 time: 994.4949586391449



epoch: 4 lr: 0.32 train loss: 0.5002247698974609 train acc: 0.82538 val loss: 1.4085215454101563 val acc: 0.6339 time: 1329.4532070159912



epoch: 5 lr: 0.4 train loss: 0.42898507537841796 train acc: 0.85166 val loss: 1.0110733062744142 val acc: 0.6936 time: 1692.034119606018


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.014433642601966858 train acc: 0.99588 val loss: 0.3712515808105469 val acc: 0.9079 time: 6473.598547458649



epoch: 21 lr: 0.026000000000000002 train loss: 0.012112502950429916 train acc: 0.99662 val loss: 0.3742081428527832 val acc: 0.9074 time: 6831.260121583939



epoch: 22 lr: 0.019000000000000003 train loss: 0.009388836225569248 train acc: 0.99772 val loss: 0.36701606674194337 val acc: 0.9095 time: 7160.159880161285



epoch: 23 lr: 0.012 train loss: 0.007516123212873936 train acc: 0.99828 val loss: 0.3687893196105957 val acc: 0.9091 time: 7483.458249330521



epoch: 24 lr: 0.005 train loss: 0.01032744450300932 train acc: 0.99736 val loss: 0.37061681213378905 val acc: 0.9087 time: 7810.0663385391235


### Augmentation inside network
Augmentation Appled at: After 1st convolution  
Augmentation Strategy: Random Pad Crop (pad=1,cutSize=8)

In [0]:
pad=1
cutSize=8
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch   

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.458164061279297 train acc: 0.46978 val loss: 1.0126893096923828 val acc: 0.6366 time: 312.94167137145996



epoch: 2 lr: 0.16 train loss: 0.7973662045288086 train acc: 0.71806 val loss: 0.9206479278564453 val acc: 0.6877 time: 619.7380905151367



epoch: 3 lr: 0.24 train loss: 0.6153512570190429 train acc: 0.78678 val loss: 0.7740326385498046 val acc: 0.7448 time: 932.5455813407898



epoch: 4 lr: 0.32 train loss: 0.5288537127685546 train acc: 0.81496 val loss: 0.9886033142089844 val acc: 0.7004 time: 1281.8481171131134



epoch: 5 lr: 0.4 train loss: 0.4507023260498047 train acc: 0.84454 val loss: 0.6566224060058594 val acc: 0.7779 time: 1621.7052855491638


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.018307467818260193 train acc: 0.99426 val loss: 0.3601479965209961 val acc: 0.9075 time: 6513.843440055847



epoch: 21 lr: 0.026000000000000002 train loss: 0.016564615815877915 train acc: 0.99492 val loss: 0.3574476318359375 val acc: 0.9065 time: 6856.54891037941



epoch: 22 lr: 0.019000000000000003 train loss: 0.012975655930042266 train acc: 0.9962 val loss: 0.35752385940551756 val acc: 0.9087 time: 7145.695568084717



epoch: 23 lr: 0.012 train loss: 0.015529093179702759 train acc: 0.99518 val loss: 0.35403167877197267 val acc: 0.9089 time: 7472.154883861542



epoch: 24 lr: 0.005 train loss: 0.013565014469623566 train acc: 0.99614 val loss: 0.35403314895629884 val acc: 0.9091 time: 7793.047049045563


### Augmentation inside network
Augmentation Appled at: After 1st convolution  
Augmentation Strategy: Random Pad Crop (pad=2,cutsize=4)

In [0]:
pad=2
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.6665190509033203 train acc: 0.41174 val loss: 1.3188296173095704 val acc: 0.5366 time: 306.766872882843



epoch: 2 lr: 0.16 train loss: 1.1134955206298829 train acc: 0.60544 val loss: 0.9823518981933593 val acc: 0.645 time: 602.5171127319336



epoch: 3 lr: 0.24 train loss: 0.8627255871582031 train acc: 0.6958 val loss: 0.9149790283203125 val acc: 0.6778 time: 934.1883838176727



epoch: 4 lr: 0.32 train loss: 0.717933583984375 train acc: 0.74958 val loss: 0.7978549179077148 val acc: 0.7245 time: 1227.2634735107422



epoch: 5 lr: 0.4 train loss: 0.6073503503417969 train acc: 0.79008 val loss: 0.7257555465698242 val acc: 0.7465 time: 1558.592827796936


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.12287731004714966 train acc: 0.96172 val loss: 0.59318916015625 val acc: 0.8221 time: 6295.713894605637



epoch: 21 lr: 0.026000000000000002 train loss: 0.11210089860916138 train acc: 0.96538 val loss: 0.6026990142822266 val acc: 0.8194 time: 6623.849063634872



epoch: 22 lr: 0.019000000000000003 train loss: 0.1270035678100586 train acc: 0.95946 val loss: 0.5297336853027343 val acc: 0.8346 time: 6953.815659046173



epoch: 23 lr: 0.012 train loss: 0.09680546756744385 train acc: 0.96982 val loss: 0.5035704879760742 val acc: 0.8516 time: 7242.827726840973



epoch: 24 lr: 0.005 train loss: 0.08664260103225709 train acc: 0.9739 val loss: 0.502859732055664 val acc: 0.8454 time: 7568.04896402359


### Augmentation inside network
Augmentation Appled at: After 1st convolution  
Augmentation Strategy: Random Pad Crop (pad=2,cutsize=8)

In [0]:
pad=2
cutSize=8
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.6671758679199218 train acc: 0.40916 val loss: 1.3326741790771484 val acc: 0.5145 time: 308.8178584575653



epoch: 2 lr: 0.16 train loss: 1.1341986395263672 train acc: 0.5953 val loss: 1.0001943145751953 val acc: 0.6484 time: 626.3598575592041



epoch: 3 lr: 0.24 train loss: 0.8928193743896484 train acc: 0.68358 val loss: 0.8606158950805664 val acc: 0.6973 time: 954.7900516986847



epoch: 4 lr: 0.32 train loss: 0.7312901742553711 train acc: 0.74674 val loss: 0.7395172241210938 val acc: 0.7407 time: 1237.4006350040436



epoch: 5 lr: 0.4 train loss: 0.6241932290649415 train acc: 0.78412 val loss: 0.6859697784423828 val acc: 0.7595 time: 1517.785236120224


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.14139594734191893 train acc: 0.95444 val loss: 0.597479817199707 val acc: 0.82 time: 6238.96818113327



epoch: 21 lr: 0.026000000000000002 train loss: 0.13843443103790284 train acc: 0.95552 val loss: 0.5362965423583984 val acc: 0.8405 time: 6547.7421452999115



epoch: 22 lr: 0.019000000000000003 train loss: 0.11942970338821411 train acc: 0.96188 val loss: 0.5366804824829101 val acc: 0.8348 time: 6854.317105054855



epoch: 23 lr: 0.012 train loss: 0.11858439964294433 train acc: 0.9622 val loss: 0.5613835708618164 val acc: 0.8286 time: 7183.854322910309



epoch: 24 lr: 0.005 train loss: 0.11282662817001343 train acc: 0.96382 val loss: 0.5903560653686524 val acc: 0.8241 time: 7499.541564464569


### Augmentation inside network
Augmentation Appled at: After 1st convolution  
Augmentation Strategy: Random Pad Crop (pad=4,cutsize=4)

In [0]:
pad=4
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.5187113116455078 train acc: 0.44504 val loss: 1.4419800170898438 val acc: 0.5321 time: 335.9128267765045



epoch: 2 lr: 0.16 train loss: 0.8664142315673828 train acc: 0.69084 val loss: 0.914839013671875 val acc: 0.6949 time: 654.8884637355804



epoch: 3 lr: 0.24 train loss: 0.6443114953613281 train acc: 0.7744 val loss: 0.8579335800170899 val acc: 0.7074 time: 948.1267635822296



epoch: 4 lr: 0.32 train loss: 0.5481588903808594 train acc: 0.81108 val loss: 0.8897675415039062 val acc: 0.7215 time: 1278.2319159507751



epoch: 5 lr: 0.4 train loss: 0.47748458526611326 train acc: 0.8356 val loss: 0.8229106842041015 val acc: 0.7517 time: 1564.429339647293


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.0294507517015934 train acc: 0.99112 val loss: 0.3231544403076172 val acc: 0.9121 time: 6321.427857875824



epoch: 21 lr: 0.026000000000000002 train loss: 0.02961469107091427 train acc: 0.99052 val loss: 0.3180964477539063 val acc: 0.9128 time: 6636.2335550785065



epoch: 22 lr: 0.019000000000000003 train loss: 0.02613851903319359 train acc: 0.9917 val loss: 0.3181461250305176 val acc: 0.913 time: 6962.006323099136



epoch: 23 lr: 0.012 train loss: 0.026843778510093688 train acc: 0.99166 val loss: 0.31983998565673827 val acc: 0.9149 time: 7290.126756906509



epoch: 24 lr: 0.005 train loss: 0.026478399572372437 train acc: 0.99202 val loss: 0.3156652931213379 val acc: 0.9145 time: 7613.910309314728


### Augmentation inside network
Augmentation Appled at: After 1st convolution  
Augmentation Strategy: Random Pad Crop (pad=4,cutsize=8)

In [0]:
pad=4
cutSize=8
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.data_aug2(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.513200711669922 train acc: 0.44656 val loss: 1.286187078857422 val acc: 0.5724 time: 303.1924624443054



epoch: 2 lr: 0.16 train loss: 0.8872401934814453 train acc: 0.68434 val loss: 1.351294024658203 val acc: 0.5962 time: 613.5565748214722



epoch: 3 lr: 0.24 train loss: 0.6717058825683594 train acc: 0.76646 val loss: 0.9458463623046875 val acc: 0.7094 time: 920.2210185527802



epoch: 4 lr: 0.32 train loss: 0.5733397882080078 train acc: 0.79958 val loss: 0.6886352966308594 val acc: 0.7766 time: 1239.274215221405



epoch: 5 lr: 0.4 train loss: 0.4983666015625 train acc: 0.82746 val loss: 1.1944845336914063 val acc: 0.6603 time: 1538.3806099891663


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.039879504640102384 train acc: 0.98694 val loss: 0.322952473449707 val acc: 0.9126 time: 6213.5818157196045



epoch: 21 lr: 0.026000000000000002 train loss: 0.04296023806333542 train acc: 0.9857 val loss: 0.32143545989990235 val acc: 0.9128 time: 6531.505244731903



epoch: 22 lr: 0.019000000000000003 train loss: 0.04071170263051987 train acc: 0.98706 val loss: 0.3218938507080078 val acc: 0.9129 time: 6843.077108383179



epoch: 23 lr: 0.012 train loss: 0.04040976460814476 train acc: 0.98706 val loss: 0.31473096923828126 val acc: 0.9145 time: 7167.435603380203



epoch: 24 lr: 0.005 train loss: 0.0369829578435421 train acc: 0.98822 val loss: 0.3137911018371582 val acc: 0.9156 time: 7474.438971281052


### Augmentation inside network
Augmentation Appled at: After ResNet block 1  
Augmentation Strategy: Random Pad Crop (pad=1, cutSize=4)

In [0]:
pad=1
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.data_aug2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.6739215759277344 train acc: 0.40466 val loss: 1.286931475830078 val acc: 0.5357 time: 294.3380506038666



epoch: 2 lr: 0.16 train loss: 1.1258365887451172 train acc: 0.59782 val loss: 1.0048131561279297 val acc: 0.6437 time: 628.3957533836365



epoch: 3 lr: 0.24 train loss: 0.8804691302490234 train acc: 0.68812 val loss: 0.8588029602050781 val acc: 0.701 time: 948.2764320373535



epoch: 4 lr: 0.32 train loss: 0.729170259399414 train acc: 0.74394 val loss: 0.7886862258911133 val acc: 0.7235 time: 1275.215219259262



epoch: 5 lr: 0.4 train loss: 0.610307648010254 train acc: 0.78756 val loss: 0.8139519317626953 val acc: 0.7198 time: 1582.7947027683258


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.1190552806854248 train acc: 0.96178 val loss: 0.533566079711914 val acc: 0.8421 time: 6342.409291028976



epoch: 21 lr: 0.026000000000000002 train loss: 0.0958775252532959 train acc: 0.97042 val loss: 0.623066682434082 val acc: 0.8173 time: 6638.141429901123



epoch: 22 lr: 0.019000000000000003 train loss: 0.10080661396026612 train acc: 0.96834 val loss: 0.6682453765869141 val acc: 0.8139 time: 6951.047869920731



epoch: 23 lr: 0.012 train loss: 0.08566309440612793 train acc: 0.97266 val loss: 0.5962306350708008 val acc: 0.827 time: 7251.440149784088



epoch: 24 lr: 0.005 train loss: 0.08281389841079712 train acc: 0.97444 val loss: 0.8060619613647461 val acc: 0.7816 time: 7564.801788806915


### Augmentation inside network
Augmentation Appled at: After ResNet block 1  
Augmentation Strategy: Random Pad Crop (pad=1, cutSize=8)

In [0]:
pad=1
cutSize=8
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.data_aug2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.7417633923339844 train acc: 0.37364 val loss: 1.4205133575439453 val acc: 0.4753 time: 362.96311473846436



epoch: 2 lr: 0.16 train loss: 1.221131226196289 train acc: 0.56084 val loss: 1.0864014678955078 val acc: 0.6079 time: 652.9500730037689



epoch: 3 lr: 0.24 train loss: 0.9776904449462891 train acc: 0.6505 val loss: 0.9139682312011719 val acc: 0.6768 time: 965.6120913028717



epoch: 4 lr: 0.32 train loss: 0.8347844329833984 train acc: 0.70426 val loss: 0.9207713226318359 val acc: 0.6778 time: 1286.5324108600616



epoch: 5 lr: 0.4 train loss: 0.7412512194824219 train acc: 0.73696 val loss: 0.8405061187744141 val acc: 0.7032 time: 1629.4906277656555


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.22528325637817384 train acc: 0.9251 val loss: 0.6059334396362305 val acc: 0.808 time: 6393.437779188156



epoch: 21 lr: 0.026000000000000002 train loss: 0.19785403045654296 train acc: 0.9346 val loss: 0.5388290817260742 val acc: 0.8298 time: 6686.985191822052



epoch: 22 lr: 0.019000000000000003 train loss: 0.18326292053222656 train acc: 0.94054 val loss: 0.5831904495239257 val acc: 0.8201 time: 6972.777536869049



epoch: 23 lr: 0.012 train loss: 0.20300742404937744 train acc: 0.93142 val loss: 0.5215156661987305 val acc: 0.8363 time: 7303.324038267136



epoch: 24 lr: 0.005 train loss: 0.2001133811569214 train acc: 0.93176 val loss: 0.6801018936157227 val acc: 0.8033 time: 7634.223736524582


### Augmentation inside network
Augmentation Appled at: After ResNet block 1  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=4


In [0]:
pad=2
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.data_aug2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.6936198791503907 train acc: 0.39504 val loss: 1.3851702270507813 val acc: 0.4816 time: 351.40392565727234



epoch: 2 lr: 0.16 train loss: 1.1764398583984375 train acc: 0.58058 val loss: 1.020491323852539 val acc: 0.6382 time: 674.0564365386963



epoch: 3 lr: 0.24 train loss: 0.9189568292236329 train acc: 0.67274 val loss: 0.8705541320800781 val acc: 0.6879 time: 985.7614064216614



epoch: 4 lr: 0.32 train loss: 0.7653041217041016 train acc: 0.72922 val loss: 0.8306340744018554 val acc: 0.7026 time: 1301.734842300415



epoch: 5 lr: 0.4 train loss: 0.651315456237793 train acc: 0.77382 val loss: 0.7953774230957031 val acc: 0.7211 time: 1631.6571145057678


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.1381177857208252 train acc: 0.95532 val loss: 0.6125480285644531 val acc: 0.8168 time: 6382.351236343384



epoch: 21 lr: 0.026000000000000002 train loss: 0.13153160533905028 train acc: 0.95698 val loss: 0.6833514389038086 val acc: 0.801 time: 6703.561769485474



epoch: 22 lr: 0.019000000000000003 train loss: 0.12263097290039063 train acc: 0.9606 val loss: 0.5654070877075196 val acc: 0.8368 time: 7005.043070793152



epoch: 23 lr: 0.012 train loss: 0.12072544479370118 train acc: 0.96022 val loss: 0.5980527084350586 val acc: 0.8277 time: 7324.938195705414



epoch: 24 lr: 0.005 train loss: 0.11084938835144043 train acc: 0.96386 val loss: 0.6247633514404297 val acc: 0.8209 time: 7647.524335384369


### Augmentation inside network
Augmentation Appled at: After ResNet block 1  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=8


In [0]:
pad=2
cutSize=8
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.data_aug2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.56013619140625 train acc: 0.42906 val loss: 2.167406573486328 val acc: 0.4234 time: 344.6038703918457



epoch: 2 lr: 0.16 train loss: 0.965200517578125 train acc: 0.65212 val loss: 1.076610043334961 val acc: 0.6574 time: 690.1129419803619



epoch: 3 lr: 0.24 train loss: 0.7549077197265625 train acc: 0.73518 val loss: 1.7094183288574218 val acc: 0.5879 time: 989.68430352211



epoch: 4 lr: 0.32 train loss: 0.6578508767700195 train acc: 0.77156 val loss: 0.6544062484741211 val acc: 0.7791 time: 1289.2147526741028



epoch: 5 lr: 0.4 train loss: 0.5648462893676758 train acc: 0.80576 val loss: 0.640186262512207 val acc: 0.7943 time: 1588.2952919006348


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.07266371158599853 train acc: 0.97594 val loss: 0.307125821685791 val acc: 0.9126 time: 6481.260657310486



epoch: 21 lr: 0.026000000000000002 train loss: 0.06529813711166382 train acc: 0.97796 val loss: 0.3039977523803711 val acc: 0.9174 time: 6814.184375524521



epoch: 22 lr: 0.019000000000000003 train loss: 0.06450075716018677 train acc: 0.97856 val loss: 0.3092188621520996 val acc: 0.9154 time: 7158.5696449279785



epoch: 23 lr: 0.012 train loss: 0.05043654715776443 train acc: 0.98334 val loss: 0.2976011734008789 val acc: 0.9192 time: 7450.325302600861



epoch: 24 lr: 0.005 train loss: 0.05761500784635544 train acc: 0.98092 val loss: 0.30888068771362304 val acc: 0.9194 time: 7787.095217704773


### Augmentation inside network
Augmentation Appled at: After ResNet block 1  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=2


In [0]:
pad=2
cutSize=2
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch  

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.data_aug2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.681621817626953 train acc: 0.40154 val loss: 1.349463589477539 val acc: 0.5132 time: 278.8710696697235



epoch: 2 lr: 0.16 train loss: 1.1535218408203125 train acc: 0.587 val loss: 1.058209457397461 val acc: 0.6206 time: 557.1495904922485



epoch: 3 lr: 0.24 train loss: 0.9144048077392578 train acc: 0.674 val loss: 0.8521651641845703 val acc: 0.6957 time: 880.1622431278229



epoch: 4 lr: 0.32 train loss: 0.7642893084716796 train acc: 0.73104 val loss: 0.8233490661621093 val acc: 0.7105 time: 1197.211720943451



epoch: 5 lr: 0.4 train loss: 0.6539117254638672 train acc: 0.77204 val loss: 0.766605207824707 val acc: 0.7237 time: 1485.4809234142303


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.13212342210769654 train acc: 0.95744 val loss: 0.6826962219238282 val acc: 0.8067 time: 6239.917277812958



epoch: 21 lr: 0.026000000000000002 train loss: 0.11413112407684327 train acc: 0.96404 val loss: 0.6130329574584961 val acc: 0.821 time: 6525.923155546188



epoch: 22 lr: 0.019000000000000003 train loss: 0.1229829419708252 train acc: 0.95982 val loss: 0.5516336563110351 val acc: 0.8337 time: 6824.373297452927



epoch: 23 lr: 0.012 train loss: 0.10450569353103638 train acc: 0.96646 val loss: 0.6602913742065429 val acc: 0.8145 time: 7145.423587560654



epoch: 24 lr: 0.005 train loss: 0.09760386112213135 train acc: 0.96862 val loss: 0.5850477844238281 val acc: 0.8396 time: 7460.729785442352


### Augmentation inside network
Augmentation Appled at: After 2nd Resnet Block  
Augmentation Strategy: Random Pad Crop (pad=1,cutSize=2)

In [0]:
pad=1
cutSize=2
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch    

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.data_aug2(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.7001617083740235 train acc: 0.3922 val loss: 1.339414193725586 val acc: 0.5143 time: 299.69847774505615



epoch: 2 lr: 0.16 train loss: 1.1771773071289062 train acc: 0.57568 val loss: 1.040238494873047 val acc: 0.6246 time: 598.1110906600952



epoch: 3 lr: 0.24 train loss: 0.9128782604980469 train acc: 0.67452 val loss: 0.9061826934814453 val acc: 0.6762 time: 887.5305528640747



epoch: 4 lr: 0.32 train loss: 0.7596427526855469 train acc: 0.73126 val loss: 0.8658771270751953 val acc: 0.6931 time: 1187.039624452591



epoch: 5 lr: 0.4 train loss: 0.6420351281738281 train acc: 0.77646 val loss: 0.6830802795410156 val acc: 0.7627 time: 1508.1716122627258


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.09022929483413697 train acc: 0.97062 val loss: 0.8588045776367188 val acc: 0.7794 time: 6146.781136512756



epoch: 21 lr: 0.026000000000000002 train loss: 0.07202292917251588 train acc: 0.97752 val loss: 0.9430787750244141 val acc: 0.7693 time: 6446.616165399551



epoch: 22 lr: 0.019000000000000003 train loss: 0.0692700567150116 train acc: 0.97806 val loss: 1.1475429138183593 val acc: 0.729 time: 6764.447984933853



epoch: 23 lr: 0.012 train loss: 0.0592445977973938 train acc: 0.98094 val loss: 1.122024041748047 val acc: 0.7434 time: 7088.589679718018



epoch: 24 lr: 0.005 train loss: 0.061602747039794924 train acc: 0.98076 val loss: 1.58162099609375 val acc: 0.6831 time: 7419.569298744202


### Augmentation inside network
Augmentation Appled at: After 2nd Resnet Block  
Augmentation Strategy: Random Pad Crop (pad=1,cutSize=4)

In [0]:
pad=1
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch    

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.data_aug2(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.5853421716308593 train acc: 0.41802 val loss: 1.358955926513672 val acc: 0.5566 time: 386.5805432796478



epoch: 2 lr: 0.16 train loss: 0.9566800061035157 train acc: 0.65616 val loss: 1.0506786956787109 val acc: 0.6646 time: 690.7436211109161



epoch: 3 lr: 0.24 train loss: 0.7710095367431641 train acc: 0.72892 val loss: 0.8113151550292969 val acc: 0.7305 time: 1043.1320261955261



epoch: 4 lr: 0.32 train loss: 0.6450969400024414 train acc: 0.77518 val loss: 1.0972739318847655 val acc: 0.6964 time: 1384.8567266464233



epoch: 5 lr: 0.4 train loss: 0.5720924963378906 train acc: 0.80236 val loss: 0.6103920150756836 val acc: 0.7946 time: 1725.2505073547363


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.04070400715827942 train acc: 0.98734 val loss: 0.38150966796875 val acc: 0.9051 time: 6732.285389661789



epoch: 21 lr: 0.026000000000000002 train loss: 0.032720000525712965 train acc: 0.98952 val loss: 0.39148032455444337 val acc: 0.9094 time: 7040.745693206787



epoch: 22 lr: 0.019000000000000003 train loss: 0.033570353409051894 train acc: 0.9889 val loss: 0.3844665252685547 val acc: 0.9062 time: 7360.630423784256



epoch: 23 lr: 0.012 train loss: 0.02908505045413971 train acc: 0.99058 val loss: 0.37829129943847656 val acc: 0.9079 time: 7669.931704521179



epoch: 24 lr: 0.005 train loss: 0.02618369418203831 train acc: 0.99184 val loss: 0.369765153503418 val acc: 0.9102 time: 7981.225943803787


### Augmentation inside network
Augmentation Appled at: After 2nd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=1

In [0]:
pad=2
cutSize=1
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad, cutSize=cutSize)
    else:    
      return batch   

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.data_aug2(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.7440642529296875 train acc: 0.37016 val loss: 1.3235881774902343 val acc: 0.5149 time: 331.13750195503235



epoch: 2 lr: 0.16 train loss: 1.2021481072998046 train acc: 0.5652 val loss: 1.042958135986328 val acc: 0.6245 time: 631.006489276886



epoch: 3 lr: 0.24 train loss: 0.9656727734375 train acc: 0.6541 val loss: 0.9025577514648437 val acc: 0.6709 time: 983.6305432319641



epoch: 4 lr: 0.32 train loss: 0.8030006311035156 train acc: 0.71528 val loss: 1.0826359649658204 val acc: 0.6282 time: 1316.047456741333



epoch: 5 lr: 0.4 train loss: 0.680142325744629 train acc: 0.7622 val loss: 0.7625468200683594 val acc: 0.7322 time: 1612.1160216331482


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.1048877684211731 train acc: 0.96536 val loss: 0.9587921401977539 val acc: 0.7572 time: 6370.264518022537



epoch: 21 lr: 0.026000000000000002 train loss: 0.0878136919784546 train acc: 0.9715 val loss: 1.5601193420410155 val acc: 0.6761 time: 6696.578418970108



epoch: 22 lr: 0.019000000000000003 train loss: 0.078529844789505 train acc: 0.97476 val loss: 1.9059832458496093 val acc: 0.6555 time: 7017.565333366394



epoch: 23 lr: 0.012 train loss: 0.07627339252471924 train acc: 0.9754 val loss: 1.166117562866211 val acc: 0.7372 time: 7312.741885900497



epoch: 24 lr: 0.005 train loss: 0.07019679019927978 train acc: 0.97716 val loss: 1.4624857940673828 val acc: 0.7026 time: 7637.79608130455


### Augmentation inside network
Augmentation Appled at: After 2nd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=2

In [0]:
pad=2
cutSize=2
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad, cutSize=cutSize)
    else:    
      return batch   

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.data_aug2(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.5699828369140625 train acc: 0.42164 val loss: 1.4111896392822265 val acc: 0.5401 time: 359.60451221466064



epoch: 2 lr: 0.16 train loss: 0.9010613348388672 train acc: 0.67792 val loss: 1.5593408813476564 val acc: 0.5732 time: 691.2611236572266



epoch: 3 lr: 0.24 train loss: 0.6858496139526368 train acc: 0.76004 val loss: 0.8301117980957031 val acc: 0.7208 time: 1017.5803172588348



epoch: 4 lr: 0.32 train loss: 0.5677341485595703 train acc: 0.80348 val loss: 0.914662255859375 val acc: 0.7237 time: 1336.6949391365051



epoch: 5 lr: 0.4 train loss: 0.49014785888671875 train acc: 0.83036 val loss: 2.1888024047851564 val acc: 0.5737 time: 1678.2270486354828


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.012502557260990144 train acc: 0.99622 val loss: 0.4685919906616211 val acc: 0.9032 time: 6494.327630996704



epoch: 21 lr: 0.026000000000000002 train loss: 0.011852895233929158 train acc: 0.99632 val loss: 0.47170534057617186 val acc: 0.9008 time: 6803.415456056595



epoch: 22 lr: 0.019000000000000003 train loss: 0.010550170094370842 train acc: 0.99696 val loss: 0.45179154663085935 val acc: 0.9046 time: 7113.258825778961



epoch: 23 lr: 0.012 train loss: 0.011364735050201417 train acc: 0.99652 val loss: 0.4750121398925781 val acc: 0.9046 time: 7441.08469581604



epoch: 24 lr: 0.005 train loss: 0.008613952351510525 train acc: 0.99744 val loss: 0.46613668670654296 val acc: 0.9038 time: 7772.370632648468


### Augmentation inside network
Augmentation Appled at: After 2nd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=4

In [0]:
pad=2
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad, cutSize=cutSize)
    else:    
      return batch   

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.data_aug2(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.8136161712646484 train acc: 0.3405 val loss: 1.4393283813476563 val acc: 0.4635 time: 344.3578281402588



epoch: 2 lr: 0.16 train loss: 1.3118828790283203 train acc: 0.5239 val loss: 1.2169591522216796 val acc: 0.5473 time: 656.7080726623535



epoch: 3 lr: 0.24 train loss: 1.0834223120117188 train acc: 0.61122 val loss: 1.077284375 val acc: 0.6134 time: 988.170604467392



epoch: 4 lr: 0.32 train loss: 0.9143649081420898 train acc: 0.6733 val loss: 0.8646190643310547 val acc: 0.6961 time: 1294.3894147872925



epoch: 5 lr: 0.4 train loss: 0.793438695678711 train acc: 0.71972 val loss: 0.8408568817138672 val acc: 0.7037 time: 1615.2668969631195


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.23353861396789552 train acc: 0.91892 val loss: 1.1219321685791015 val acc: 0.7376 time: 6499.231781721115



epoch: 21 lr: 0.026000000000000002 train loss: 0.2400260569000244 train acc: 0.91724 val loss: 0.9329610504150391 val acc: 0.7595 time: 6864.624772071838



epoch: 22 lr: 0.019000000000000003 train loss: 0.20351605892181396 train acc: 0.9298 val loss: 1.1155522521972656 val acc: 0.7364 time: 7197.193012475967



epoch: 23 lr: 0.012 train loss: 0.1783711735534668 train acc: 0.93966 val loss: 0.8108752380371094 val acc: 0.789 time: 7506.890388727188



epoch: 24 lr: 0.005 train loss: 0.1894309169769287 train acc: 0.93496 val loss: 1.2548774505615234 val acc: 0.7123 time: 7863.731426000595


### Augmentation inside network
Augmentation Appled at: After 3rd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=1,cutSize=1

In [0]:
pad=1
cutSize=1
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch 

  def call(self, x, y):
    h = self.pool(self.blk4(self.data_aug2(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.7425555780029296 train acc: 0.37724 val loss: 1.3972201293945312 val acc: 0.4872 time: 314.78137254714966



epoch: 2 lr: 0.16 train loss: 1.1797134692382814 train acc: 0.58026 val loss: 1.0741511077880859 val acc: 0.6126 time: 650.1410677433014



epoch: 3 lr: 0.24 train loss: 0.9001570202636718 train acc: 0.6829 val loss: 0.9994837921142579 val acc: 0.6354 time: 991.2281901836395



epoch: 4 lr: 0.32 train loss: 0.7089501928710937 train acc: 0.754 val loss: 0.9240078216552734 val acc: 0.6751 time: 1310.907786846161



epoch: 5 lr: 0.4 train loss: 0.5627576852416992 train acc: 0.80846 val loss: 0.7926905624389649 val acc: 0.7284 time: 1615.0288844108582


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.002298788585662842 train acc: 1.0 val loss: 0.7047800231933594 val acc: 0.8201 time: 6414.562806606293



epoch: 21 lr: 0.026000000000000002 train loss: 0.002134357281923294 train acc: 1.0 val loss: 0.7026110488891602 val acc: 0.819 time: 6742.912936925888



epoch: 22 lr: 0.019000000000000003 train loss: 0.0018622981816530228 train acc: 1.0 val loss: 0.6964565673828125 val acc: 0.8187 time: 7049.991405010223



epoch: 23 lr: 0.012 train loss: 0.0017173825323581696 train acc: 1.0 val loss: 0.7065638732910157 val acc: 0.8216 time: 7357.684489250183



epoch: 24 lr: 0.005 train loss: 0.0016665874671936034 train acc: 1.0 val loss: 0.7127612518310547 val acc: 0.8188 time: 7666.398751974106


### Augmentation inside network
Augmentation Appled at: After 3rd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=1,cutSize=2

In [0]:
pad=1
cutSize=2
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch 

  def call(self, x, y):
    h = self.pool(self.blk4(self.data_aug2(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

epoch: 1 lr: 0.08 train loss: 1.5618844573974608 train acc: 0.42312 val loss: 1.4428502960205078 val acc: 0.5443 time: 327.4184856414795



epoch: 2 lr: 0.16 train loss: 0.8504948211669922 train acc: 0.69358 val loss: 0.9592930053710937 val acc: 0.7016 time: 633.9718420505524



epoch: 3 lr: 0.24 train loss: 0.6257404537963868 train acc: 0.78086 val loss: 1.0131699584960938 val acc: 0.7188 time: 948.2642438411713



epoch: 4 lr: 0.32 train loss: 0.49548582733154295 train acc: 0.8285 val loss: 1.1664123779296875 val acc: 0.6816 time: 1261.5304386615753



epoch: 5 lr: 0.4 train loss: 0.4180797412109375 train acc: 0.85646 val loss: 0.7690294967651368 val acc: 0.7744 time: 1605.7766950130463


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.0014309308955445885 train acc: 0.99964 val loss: 0.6868367446899414 val acc: 0.884 time: 6409.046169519424



epoch: 21 lr: 0.026000000000000002 train loss: 0.0012175669299811125 train acc: 0.9997 val loss: 0.6830407089233398 val acc: 0.8851 time: 6705.023818254471



epoch: 22 lr: 0.019000000000000003 train loss: 0.0008372579548507928 train acc: 0.99984 val loss: 0.6648986877441406 val acc: 0.8854 time: 7024.939955234528



epoch: 23 lr: 0.012 train loss: 0.000932557633407414 train acc: 0.9998 val loss: 0.6990473846435546 val acc: 0.8851 time: 7353.696537017822



epoch: 24 lr: 0.005 train loss: 0.0007508684581145644 train acc: 0.99988 val loss: 0.699462158203125 val acc: 0.8855 time: 7675.524048805237


### Augmentation inside network
Augmentation Appled at: After 3rd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=1

In [0]:
pad=2
cutSize=1
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch 

  def call(self, x, y):
    h = self.pool(self.blk4(self.data_aug2(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.8231514172363281 train acc: 0.33814 val loss: 1.442267401123047 val acc: 0.4613 time: 289.6646785736084



epoch: 2 lr: 0.16 train loss: 1.2966751123046876 train acc: 0.53362 val loss: 1.2198311065673828 val acc: 0.5684 time: 609.1216335296631



epoch: 3 lr: 0.24 train loss: 1.0168110852050782 train acc: 0.64206 val loss: 0.9677573272705078 val acc: 0.6539 time: 946.1997685432434



epoch: 4 lr: 0.32 train loss: 0.8102151486206055 train acc: 0.71464 val loss: 0.882210791015625 val acc: 0.6835 time: 1258.6346690654755



epoch: 5 lr: 0.4 train loss: 0.657751589050293 train acc: 0.77368 val loss: 0.7634411819458008 val acc: 0.731 time: 1550.4228093624115


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.009628053575754166 train acc: 0.99906 val loss: 0.8504127410888672 val acc: 0.7917 time: 6273.854773283005



epoch: 21 lr: 0.026000000000000002 train loss: 0.0065399713027477265 train acc: 0.99952 val loss: 0.8535119537353516 val acc: 0.7986 time: 6571.15731549263



epoch: 22 lr: 0.019000000000000003 train loss: 0.005081442323923111 train acc: 0.99962 val loss: 0.8374009582519532 val acc: 0.7963 time: 6858.866739749908



epoch: 23 lr: 0.012 train loss: 0.004174236983656883 train acc: 0.99996 val loss: 0.7926763214111329 val acc: 0.805 time: 7174.6339502334595



epoch: 24 lr: 0.005 train loss: 0.002823784787654877 train acc: 0.99994 val loss: 0.7744853057861328 val acc: 0.8105 time: 7478.011656522751


### Augmentation inside network
Augmentation Appled at: After 3rd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=2

In [0]:
pad=2
cutSize=2
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch 

  def call(self, x, y):
    h = self.pool(self.blk4(self.data_aug2(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.8160484887695312 train acc: 0.3489 val loss: 1.4179361694335937 val acc: 0.4607 time: 295.72327518463135



epoch: 2 lr: 0.16 train loss: 1.3301858636474608 train acc: 0.52034 val loss: 1.1543548431396484 val acc: 0.5706 time: 597.4277584552765



epoch: 3 lr: 0.24 train loss: 1.0542542346191406 train acc: 0.62694 val loss: 0.9825055938720703 val acc: 0.6487 time: 930.5684466362



epoch: 4 lr: 0.32 train loss: 0.8620135485839844 train acc: 0.69728 val loss: 0.9976129333496093 val acc: 0.6483 time: 1269.125559091568



epoch: 5 lr: 0.4 train loss: 0.7095697296142578 train acc: 0.75462 val loss: 0.8873738189697266 val acc: 0.6979 time: 1584.3024711608887


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.01939874532222748 train acc: 0.99566 val loss: 0.9407024597167969 val acc: 0.7874 time: 6288.347442626953



epoch: 21 lr: 0.026000000000000002 train loss: 0.019704329578876495 train acc: 0.99552 val loss: 1.0317566680908203 val acc: 0.7664 time: 6582.805755376816



epoch: 22 lr: 0.019000000000000003 train loss: 0.012853345379829406 train acc: 0.99702 val loss: 0.8391115295410156 val acc: 0.7989 time: 6875.430192708969



epoch: 23 lr: 0.012 train loss: 0.017622909216284753 train acc: 0.99514 val loss: 0.8445834136962891 val acc: 0.8063 time: 7203.672232866287



epoch: 24 lr: 0.005 train loss: 0.013363480752706528 train acc: 0.99656 val loss: 0.8369158813476563 val acc: 0.8089 time: 7539.292580127716


### Augmentation inside network
Augmentation Appled at: After 3rd Resnet Block  
Augmentation Strategy: Random Pad Crop pad=2,cutSize=4

In [0]:
pad=2
cutSize=4
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight
    self.data_aug2 = lambda x: (self.augment(x)) 
  
  def augment(self,batch):    
    if training:
      return augmentDictChannel(batch,padding=pad,cutSize=cutSize)
    else:    
      return batch 

  def call(self, x, y):
    h = self.pool(self.blk4(self.data_aug2(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

model = DavidNet()
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  training=True
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  training=False
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 2.101784052734375 train acc: 0.21598 val loss: 1.6178125579833984 val acc: 0.4219 time: 309.32568979263306



epoch: 2 lr: 0.16 train loss: 1.8587495947265624 train acc: 0.2993 val loss: 1.5619397552490235 val acc: 0.4682 time: 625.7331986427307



epoch: 3 lr: 0.24 train loss: 1.711361294555664 train acc: 0.35638 val loss: 1.7987959777832032 val acc: 0.5005 time: 930.9837007522583



epoch: 4 lr: 0.32 train loss: 1.5483372924804688 train acc: 0.41706 val loss: 1.039345263671875 val acc: 0.632 time: 1221.2258667945862



epoch: 5 lr: 0.4 train loss: 1.651786128540039 train acc: 0.37214 val loss: 1.0984959533691405 val acc: 0.6484 time: 1563.8383326530457


...


epoch: 20 lr: 0.03300000000000001 train loss: 1.0048241163635254 train acc: 0.61474 val loss: 1.073508563232422 val acc: 0.7185 time: 6350.1826910972595



epoch: 21 lr: 0.026000000000000002 train loss: 0.9919131321716309 train acc: 0.61846 val loss: 0.9957437072753906 val acc: 0.7323 time: 6633.752651929855



epoch: 22 lr: 0.019000000000000003 train loss: 1.0913423049545288 train acc: 0.57872 val loss: 1.6057800079345703 val acc: 0.6483 time: 6953.001893758774



epoch: 23 lr: 0.012 train loss: 1.2155848331069947 train acc: 0.52984 val loss: 3.0441604614257813 val acc: 0.58 time: 7273.564126491547



epoch: 24 lr: 0.005 train loss: 1.0722630225753784 train acc: 0.58592 val loss: 0.7727822174072265 val acc: 0.7826 time: 7573.882131814957
